# Train Faster RCNN Models

This notebook can be used to train Faster RCNN models.

## Configuration

In [1]:
starting_config = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
data_dir = "../../data/models/detectron2/no-augmentations"
output_dir = "runs/train/no-augmentations"
train_data = "train_data"
valid_data = "valid_data"
test_data = "test_data"
final_model_file = "model_final.pth"
n_classes = 1 # litter
batch_size_per_image = 16 # maximise for hardware
images_per_batch = 4 # maximise for hardware
learning_rate = 0.001
warmup_iters = 1000
iters = 1500
eval_iters = 300
steps = [] # (1000, 1500)
confidence_threshold = 0.3

# Detectron2 FRCNN Object Detection

## Setup

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger

setup_logger()

register_coco_instances(train_data, {}, f"{data_dir}/train/_annotations.coco.json", f"{data_dir}/train")
register_coco_instances(valid_data, {}, f"{data_dir}/valid/_annotations.coco.json", f"{data_dir}/valid")
register_coco_instances(test_data, {}, f"{data_dir}/test/_annotations.coco.json", f"{data_dir}/test")

## Train

In [ ]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    os.makedirs(output_dir, exist_ok=True)
    output_folder = output_dir
    return COCOEvaluator(dataset_name, cfg, False, output_folder)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(starting_config))
cfg.OUTPUT_DIR = output_dir

cfg.DATASETS.TRAIN = (train_data,)
cfg.DATASETS.TEST = (valid_data,)
cfg.DATALOADER.NUM_WORKERS = 1

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(starting_config)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = n_classes

cfg.SOLVER.IMS_PER_BATCH = images_per_batch
cfg.SOLVER.BASE_LR = learning_rate
cfg.SOLVER.WARMUP_ITERS = warmup_iters
cfg.SOLVER.MAX_ITER = iters
cfg.SOLVER.STEPS = steps

cfg.TEST.EVAL_PERIOD = eval_iters

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Evaluate

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, final_model_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(test_data, cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, test_data)
inference_on_dataset(trainer.model, val_loader, evaluator)

## Visualize

In [ ]:
import glob
import cv2
from detectron2.utils.visualizer import Visualizer

cfg.DATASETS.TEST = (test_data, )
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, final_model_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold

predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get(test_data)

n_images = 3
title = "Faster RCNN Litter Detection"

for imageName in glob.glob(f"{data_dir}/test/*jpg")[:n_images]:
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], metadata=test_metadata)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  img = out.get_image()[:, :, ::-1]
  cv2.imshow(title, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
